<a href="https://colab.research.google.com/github/nguyetvo/Nguyet-ML2-Online-042020/blob/master/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import feedparser as fp
import json
# import newspaper
# from newspaper import Article
from time import mktime
from datetime import datetime

In [0]:
pip install feedparser

  Stored in directory: C:\Users\Admin\AppData\Local\pip\Cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
Note: you may need to restart the kernel to use updated packages.


In [0]:
# Set the limit for number of articles to download
LIMIT = 50

In [0]:
data = {}
data['newspapers'] = {}

In [0]:
# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [0]:
count = 1

# Iterate through each news company
# the company is the name, the value is the dictionary of links
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                article['author'] = content.authors
                newsPaper['articles'].append(article)
                if count % 10 == 0:
                    print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            article['author'] = content.authors
            newsPaper['articles'].append(article)
            if count % 10 == 0: 
                print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper


# Finally it saves the articles as a JSON-file.
try:
    fname = 'scraped_articles.json'
    print('saving articles . . . in {}'.format(fname))
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
continuing...
name 'Article' is not defined
co

In [0]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [0]:
fname = 'scraped_articles.json'

In [0]:
with open(fname) as json_data:
    d = json.load(json_data)

In [0]:
for i, site in enumerate((list(d['newspapers'])[:5])):
    print(i, site)

0 newyorktimes_business
1 newyorktimes_science
2 newyorktimes_technology
3 cnnmoney
4 marketwatch_newsletters


In [0]:
# create a dataframe, with all of the articles, one row per article

for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
    else:
        new_df = pd.DataFrame.from_dict(articles)
        df = pd.concat([df, new_df], ignore_index = True)

In [0]:
df.shape

(631, 5)

In [0]:
df.head()

,author,link,published,text,title
0,"[David Streitfeld, Natasha Singer, Steven Erla...",https://www.nytimes.com/2018/03/24/technology/...,2018-03-24T20:43:21,"There are other avenues still, said Jascha Kay...",How Calls for Privacy May Upend Business for F...
1,[Emily Flitter],https://www.nytimes.com/2018/03/24/business/we...,2018-03-24T17:16:53,"And so Mr. Tran, who is in his mid-50s and spe...",The Former Khmer Rouge Slave Who Blew the Whis...
2,[Keith Bradsher],https://www.nytimes.com/2018/03/25/business/ch...,2018-03-25T15:21:56,"In that analogy, Mr. Guo will essentially beco...","China Splits Top Jobs at Central Bank, Adding ..."
3,[Ben Casselman],https://www.nytimes.com/2018/03/24/business/jo...,2018-03-25T03:00:18,News of Mr. Williams’s likely selection was fi...,A Top Candidate for New York Fed’s Leader: San...
4,[Jim Tankersley],https://www.nytimes.com/2018/03/24/business/tr...,2018-03-24T14:34:10,What is increasingly clear to many on Capitol ...,"Trump’s Tariffs Keep Allies, Markets and Indus..."


In [0]:
# list of spam sites
spam = list(d['newspapers'])[-11:]

In [0]:
# create a column for the label of the article, a label of 0 is ham, a label of 1 is spam
df['label'] = 0

In [0]:
for i, site in enumerate(spam):
    df.loc[df['link'].str.contains('{}'.format(site)), 'label'] = 1

In [0]:
# 463 non spam articles, 168 spam articles
df[['title', 'label']].groupby('label').agg('count')

,title
label,
0,463
1,168


In [0]:
# python module for cleaning!
%run cleaning_helper1.py

In [0]:
df1 = df.copy()

In [0]:
# function imported from python module
df1 = clean_column(df1, 'title', 'clean_title')

In [0]:
df1 = clean_column(df1, 'text', 'clean_text')

In [0]:
# function imported from python module
filtration(df1, 'clean_title')

In [0]:
filtration(df1, 'clean_text')

In [0]:
df1.head()

,author,link,published,text,title,label,clean_title,clean_text
0,"[David Streitfeld, Natasha Singer, Steven Erla...",https://www.nytimes.com/2018/03/24/technology/...,2018-03-24T20:43:21,"There are other avenues still, said Jascha Kay...",How Calls for Privacy May Upend Business for F...,0,calls privacy may upend business facebook google,avenues still said jascha kaykas wolff chief m...
1,[Emily Flitter],https://www.nytimes.com/2018/03/24/business/we...,2018-03-24T17:16:53,"And so Mr. Tran, who is in his mid-50s and spe...",The Former Khmer Rouge Slave Who Blew the Whis...,0,former khmer rouge slave blew whistle wells fargo,mr tran mid 50s speaks english heavy vietnames...
2,[Keith Bradsher],https://www.nytimes.com/2018/03/25/business/ch...,2018-03-25T15:21:56,"In that analogy, Mr. Guo will essentially beco...","China Splits Top Jobs at Central Bank, Adding ...",0,china splits top jobs central bank adding anot...,analogy mr guo essentially become chairman chi...
3,[Ben Casselman],https://www.nytimes.com/2018/03/24/business/jo...,2018-03-25T03:00:18,News of Mr. Williams’s likely selection was fi...,A Top Candidate for New York Fed’s Leader: San...,0,top candidate new york feds leader san franciscos,news mr williamss likely selection first repor...
4,[Jim Tankersley],https://www.nytimes.com/2018/03/24/business/tr...,2018-03-24T14:34:10,What is increasingly clear to many on Capitol ...,"Trump’s Tariffs Keep Allies, Markets and Indus...",0,trumps tariffs keep allies markets industry gu...,increasingly clear many capitol hill business ...


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [0]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [0]:
df = df1.copy()
df.head()

,author,link,published,text,title,label,clean_title,clean_text
0,"[David Streitfeld, Natasha Singer, Steven Erla...",https://www.nytimes.com/2018/03/24/technology/...,2018-03-24T20:43:21,"There are other avenues still, said Jascha Kay...",How Calls for Privacy May Upend Business for F...,0,calls privacy may upend business facebook google,avenues still said jascha kaykas wolff chief m...
1,[Emily Flitter],https://www.nytimes.com/2018/03/24/business/we...,2018-03-24T17:16:53,"And so Mr. Tran, who is in his mid-50s and spe...",The Former Khmer Rouge Slave Who Blew the Whis...,0,former khmer rouge slave blew whistle wells fargo,mr tran mid 50s speaks english heavy vietnames...
2,[Keith Bradsher],https://www.nytimes.com/2018/03/25/business/ch...,2018-03-25T15:21:56,"In that analogy, Mr. Guo will essentially beco...","China Splits Top Jobs at Central Bank, Adding ...",0,china splits top jobs central bank adding anot...,analogy mr guo essentially become chairman chi...
3,[Ben Casselman],https://www.nytimes.com/2018/03/24/business/jo...,2018-03-25T03:00:18,News of Mr. Williams’s likely selection was fi...,A Top Candidate for New York Fed’s Leader: San...,0,top candidate new york feds leader san franciscos,news mr williamss likely selection first repor...
4,[Jim Tankersley],https://www.nytimes.com/2018/03/24/business/tr...,2018-03-24T14:34:10,What is increasingly clear to many on Capitol ...,"Trump’s Tariffs Keep Allies, Markets and Indus...",0,trumps tariffs keep allies markets industry gu...,increasingly clear many capitol hill business ...


In [0]:
# Preparing the target and predictors for modeling
# Keep the title and body text separated for different models

X_body_text = df['clean_text'].values
X_title_text = df['clean_title'].values
y = df['label'].values

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df= 0.85, min_df= 0.01)

In [0]:
X_body_tfidf = tfidf.fit_transform(X_body_text)
X_title_tfidf = tfidf.fit_transform (X_title_text)

In [0]:
indices = df.index.values

In [0]:
X_body_tfidf_train, X_body_tfidf_test, \
y_body_train, y_body_test, \
indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)

In [0]:
df.loc[indices_body_train].groupby('label').agg('count')

,author,link,published,text,title,clean_title,clean_text
label,,,,,,,
0,363,363,363,363,363,363,363
1,141,141,141,141,141,141,141


In [0]:
df.loc[indices_body_test].groupby('label').agg('count')

,author,link,published,text,title,clean_title,clean_text
label,,,,,,,
0,100,100,100,100,100,100,100
1,27,27,27,27,27,27,27


In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
nb_body = MultinomialNB()

In [0]:
nb_body.fit(X_body_tfidf_train, y_body_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_body_train_pred = nb_body.predict(X_body_tfidf_train)

In [0]:
print('Naive Bayes In Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(y_body_train, y_body_train_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(y_body_train, y_body_train_pred)*100))

Naive Bayes In Sample F1 and Accuracy Scores:
F1 score 99.51%
Accuracy score 99.6%


In [0]:
np.where(y_body_train != y_body_train_pred)

(array([ 52, 449], dtype=int64),)

In [0]:
df.loc[[52, 449]]

,author,link,published,text,title,label,clean_title,clean_text
52,"[Carl Zimmer, Profiles In Science]",https://www.nytimes.com/2018/03/20/science/dav...,2018-03-20T19:41:01,These people almost entirely replaced the earl...,David Reich Unearths Human History Etched in Bone,0,david reich unearths human history etched bone,people almost entirely replaced earlier farmer...
449,[Oliver Roeder],https://fivethirtyeight.com/features/blockchai...,2018-03-07T18:13:42,The Blockchain Is Just Another Way To Make Art...,The Blockchain Is Just Another Way To Make Art...,0,blockchain another way make art money,blockchain another way make art money commerci...


In [0]:
# https://github.com/eChanClarityInsights/Fake-News-Classifier/blob/master/notebooks/03.%20Naive%20Bayes.ipynb
# # for article in dirty_df.loc[[52, 449], 'text']:
# #     print(article)
# #     print('--------------\n\n\n\n')